In [3]:
from ctmds.random_prices import rand_uniform_prices

len(rand_uniform_prices(5))

5

In [ ]:
import pytz
from datetime import datetime, timedelta
import pandas as pd

COUNTRY_ISO_TO_TIMEZONE = {
    "GB": "Europe/London",
    "FR": "Europe/Paris",
    "NL": "Europe/Amsterdam",
    "DE": "Europe/Berlin",
}

def get_country_datetime_series(date_str: str, country_code: str, granularity: str = "hourly") -> pd.DatetimeIndex:
    """
    Get the point-in-time accurate timestamps for a given date and country ISO code,
    accounting for daylight savings time transitions by detecting if a given date is a daylight
    saving transition (spring forward or fall back).

    On spring forward dates, this will result in fewer timestamps than in a normal 24 hour period.
    On fall back dates, this will result in more timestamps than in a normal 24 hour period.
    """
    if country_code not in COUNTRY_ISO_TO_TIMEZONE:
        raise ValueError("Unsupported country ISO code.")

    tz = pytz.timezone(COUNTRY_ISO_TO_TIMEZONE[country_code])

    # Convert input date to a datetime object at midnight
    dt = datetime.strptime(date_str, "%Y-%m-%d").replace(hour=0, minute=0)

    # Get the timezone naive time series
    freq = "h" if granularity == "hourly" else "30min"
    naive_range = pd.date_range(start=dt, end=dt + timedelta(days=1, seconds=-1), freq=freq)

    ## Auto-detect DST transition:
    # Localize in "strict" mode to avoid automatic DST correction
    dt_before = tz.localize(dt, is_dst=None)
    dt_after = tz.localize(dt + timedelta(days=1), is_dst=None)
    # Get UTC offset before and after transition to detect if spring forward, fall back, or neither
    offset_before = dt_before.utcoffset()
    offset_after = dt_after.utcoffset()

    # Get final timezone-aware timeseries, with proper timestamp values
    if offset_before < offset_after:
        # Spring Forward (DST Start)
        # print("spring forward")
        # Convert to timezone-aware timestamps
        aware_range = naive_range.tz_localize(tz, nonexistent="NaT")
        # Drop NaT values (times that don't exist on spring forward)
        return aware_range.dropna()
    elif offset_before > offset_after:
        # Fall Back (DST End)
        # print("fall back")
        # Combine two series, one that uses DST before transition & other uses non-DST after trans
        first_occurrence = naive_range.tz_localize(tz, ambiguous=False)
        second_occurrence = naive_range.tz_localize(tz, ambiguous=True)
        # Drop duplicates, effectively keeping the hours over the transition period
        return pd.concat(
            [first_occurrence.to_series(), second_occurrence.to_series()]
        ).drop_duplicates(keep="first").sort_values().index
    else:
        # No DST Transition
        return naive_range.tz_localize(tz)

In [ ]:
# ts = get_country_datetime_series("2024-03-31", "GB", "half-hourly")
ts = get_country_datetime_series("2024-10-27", "GB", "half-hourly")
print(len(ts))
ts

fall back
50


DatetimeIndex(['2024-10-27 00:00:00+01:00', '2024-10-27 00:30:00+01:00',
               '2024-10-27 01:00:00+01:00', '2024-10-27 01:30:00+01:00',
               '2024-10-27 01:00:00+00:00', '2024-10-27 01:30:00+00:00',
               '2024-10-27 02:00:00+00:00', '2024-10-27 02:30:00+00:00',
               '2024-10-27 03:00:00+00:00', '2024-10-27 03:30:00+00:00',
               '2024-10-27 04:00:00+00:00', '2024-10-27 04:30:00+00:00',
               '2024-10-27 05:00:00+00:00', '2024-10-27 05:30:00+00:00',
               '2024-10-27 06:00:00+00:00', '2024-10-27 06:30:00+00:00',
               '2024-10-27 07:00:00+00:00', '2024-10-27 07:30:00+00:00',
               '2024-10-27 08:00:00+00:00', '2024-10-27 08:30:00+00:00',
               '2024-10-27 09:00:00+00:00', '2024-10-27 09:30:00+00:00',
               '2024-10-27 10:00:00+00:00', '2024-10-27 10:30:00+00:00',
               '2024-10-27 11:00:00+00:00', '2024-10-27 11:30:00+00:00',
               '2024-10-27 12:00:00+00:00', '2024-1

In [118]:
[t.strftime("%Y-%m-%d %H:%M") for t in ts]

['2024-10-27 00:00',
 '2024-10-27 00:30',
 '2024-10-27 01:00',
 '2024-10-27 01:30',
 '2024-10-27 01:00',
 '2024-10-27 01:30',
 '2024-10-27 02:00',
 '2024-10-27 02:30',
 '2024-10-27 03:00',
 '2024-10-27 03:30',
 '2024-10-27 04:00',
 '2024-10-27 04:30',
 '2024-10-27 05:00',
 '2024-10-27 05:30',
 '2024-10-27 06:00',
 '2024-10-27 06:30',
 '2024-10-27 07:00',
 '2024-10-27 07:30',
 '2024-10-27 08:00',
 '2024-10-27 08:30',
 '2024-10-27 09:00',
 '2024-10-27 09:30',
 '2024-10-27 10:00',
 '2024-10-27 10:30',
 '2024-10-27 11:00',
 '2024-10-27 11:30',
 '2024-10-27 12:00',
 '2024-10-27 12:30',
 '2024-10-27 13:00',
 '2024-10-27 13:30',
 '2024-10-27 14:00',
 '2024-10-27 14:30',
 '2024-10-27 15:00',
 '2024-10-27 15:30',
 '2024-10-27 16:00',
 '2024-10-27 16:30',
 '2024-10-27 17:00',
 '2024-10-27 17:30',
 '2024-10-27 18:00',
 '2024-10-27 18:30',
 '2024-10-27 19:00',
 '2024-10-27 19:30',
 '2024-10-27 20:00',
 '2024-10-27 20:30',
 '2024-10-27 21:00',
 '2024-10-27 21:30',
 '2024-10-27 22:00',
 '2024-10-27 